# Sparkify SQL Data Modeling with Postgres

In this, we will model the data with Postgres and build an ETL pipeline using Python. The fact and dimension tables for a star database schema for a particular analytic focus is defined, and an ETL pipeline that transfers data from files in two local directories into these tables in Postgres using Python and SQL was developed.

A startup called Sparkify wants to analyze the data they've been collecting on songs and user activity on their new music streaming application. The analytics team is particularly interested in understanding what songs users are listening to. Currently, they don't have an easy way to query their data, which resides in a directory of JSON logs on user activity on the application, as well as a directory with JSON meta-data on the songs in their application.

They'd like a data engineer to create a Postgres database with tables designed to optimize queries on song play analysis. The role of this project is to create a database schema and ETL pipeline for this analysis.

We will model the data with Postgres and build an ETL pipeline using Python. The fact and dimension tables for a star database schema for a particular analytic focus is defined, and an ETL pipeline that transfers data from files in two local directories into these tables in Postgres using Python and SQL was developed.

### Songs dataset

Songs dataset is a subset of [Million Song Dataset](http://millionsongdataset.com/). Each file in the dataset is in JSON format and contains meta-data about a song and the artist of that song.

Sample record:

```json
{"num_songs": 1, "artist_id": "ARJIE2Y1187B994AB7", "artist_latitude": null, "artist_longitude": null, "artist_location": "", "artist_name": "Line Renaud", "song_id": "SOUPIRU12A6D4FA1E1", "title": "Der Kleine Dompfaff", "duration": 152.92036, "year": 0}
```

### Logs dataset

Logs dataset is generated by [Event Simulator](https://github.com/Interana/eventsim). These log files in JSON format simulate activity logs from a music streaming application based on specified configurations.

Sample record:

```json
{"artist": null, "auth": "Logged In", "firstName": "Walter", "gender": "M", "itemInSession": 0, "lastName": "Frye", "length": null, "level": "free", "location": "San Francisco-Oakland-Hayward, CA", "method": "GET","page": "Home", "registration": 1540919166796.0, "sessionId": 38, "song": null, "status": 200, "ts": 1541105830796, "userAgent": "\"Mozilla\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\/537.36 (KHTML, like Gecko) Chrome\/36.0.1985.143 Safari\/537.36\"", "userId": "39"}
```

### Quality

1. LogDataset : userID - Found users with empty string '', firstName as None
2. LogDataset : Major of the artist_id & song_id is null
3. Artists table doesn't have list of all the artists found in log
4. Songs table doesn't have all the songs found in log
5. LogDataset : Logs are ordered by timestamp, so they need to be sorted. This enables latest user level to be updated in the users table

### Tidiness

1. LogDataset : ts : timestamp column as int64 needs to converted to timestamp
2. SongPlays table : Add new column songplay_id as serial ( auto-increment )
3. user : table : Adding column ts
4. songplays : table : Adding columns itemInSession, song, artist

### Database Schema Design - Entity Relation Diagram (ERD)

The Star Database Schema used for data modeling in this ETL pipeline. There is one fact table containing all the metrics (facts) associated to each event (user actions), and four dimensions tables, containing associated information such as user name, artist name, song meta-data etc. This model enables to search the database schema with the minimum number of *SQL JOIN*s possible and enable fast read queries. The amount of data we need to analyze is not big enough to require big data solutions or NoSQL databases.

![data_schema](https://user-images.githubusercontent.com/62965911/215310998-b894e4b2-b6ba-4a2d-a84b-531e67a32cbc.png)

## Process Flow 

![](./img/process-flow.drawio.svg)

In [6]:
%%writefile requirements.txt
psycopg2-binary
python-dotenv
ipython-sql==0.4.1

Overwriting requirements.txt


In [ ]:
! pip install -r requirements.txt

In [15]:
%%writefile .env
HOST1=
USER1=
PASSWORD1=
DATABASE1=
SCHEMA1=

Overwriting .env


In [4]:
import os

from dotenv import load_dotenv
load_dotenv()

import glob
import psycopg2
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
USERNAME = os.getenv('USER1')
PASSWORD = os.getenv('PASSWORD1')
HOST = os.getenv('HOST1')
DATABASE = os.getenv('DATABASE1')
SCHEMA=os.getenv('SCHEMA1')
print(USERNAME , HOST, DATABASE, SCHEMA)

In [6]:
conn_str = 'postgresql://{0}:{1}@{2}/{3}'.format(USERNAME, PASSWORD, HOST, DATABASE)

In [7]:
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon=False
%config SqlMagic.feedback=False
%reload_ext sql
%sql {conn_str}

In [68]:
def get_conn():
    conn = psycopg2.connect(f"host={HOST} dbname={DATABASE} user={USERNAME} password={PASSWORD}")
    cur = conn.cursor()
    return cur, conn

In [69]:
cur, conn = get_conn()

In [70]:
# CREATE and SET default schema
create_schema = f"CREATE SCHEMA IF NOT EXISTS {SCHEMA}"
set_default_schema = f"SET search_path={SCHEMA}"

In [71]:
cur.execute(create_schema)
conn.commit()

cur.execute(set_default_schema)
conn.commit()

In [12]:
# CREATE TABLES QUERIES

songplay_table_create = ("""
    CREATE TABLE IF NOT EXISTS songplays (
        songplay_id SERIAL PRIMARY KEY, 
        start_time TIMESTAMP REFERENCES time (start_time), 
        user_id INT REFERENCES users (user_id), 
        level VARCHAR, 
        song_id VARCHAR REFERENCES songs (song_id), 
        artist_id VARCHAR REFERENCES artists (artist_id), 
        session_id INT, 
        location TEXT, 
        user_agent TEXT
    )
""")

user_table_create = ("""
    CREATE TABLE IF NOT EXISTS users (
        user_id INT PRIMARY KEY, 
        first_name VARCHAR, 
        last_name VARCHAR, 
        gender CHAR(1), 
        level VARCHAR
    )
""")

song_table_create = ("""
    CREATE TABLE IF NOT EXISTS songs (
        song_id VARCHAR PRIMARY KEY, 
        title VARCHAR, 
        artist_id VARCHAR REFERENCES artists (artist_id), 
        year INT, 
        duration FLOAT
    )
""")

artist_table_create = ("""
    CREATE TABLE IF NOT EXISTS artists (
        artist_id VARCHAR PRIMARY KEY, 
        name VARCHAR, 
        location TEXT , 
        latitude FLOAT , 
        longitude FLOAT 
    )
""")

time_table_create = ("""
    CREATE TABLE IF NOT EXISTS time (
        start_time TIMESTAMP PRIMARY KEY, 
        hour INT, 
        day INT, 
        week INT, 
        month INT, 
        year INT, 
        weekday VARCHAR
    )
""")

create_table_queries = [artist_table_create, song_table_create, user_table_create, time_table_create, songplay_table_create]

In [13]:
for query in create_table_queries:
    cur.execute(query)
    conn.commit()

In [25]:
!aws s3 ls s3://jan16-data/projects/sparkify/

                           PRE log_data/
                           PRE song_data/


In [ ]:
!mkdir -p data
!aws s3 sync s3://jan16-data/projects/sparkify data

create json and convert into table and load it

In [2]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files

In [ ]:
song_files = get_files('data/song_data')
filepath = song_files[0]
filepath

In [17]:
df=pd.read_json(filepath, lines=True)
df

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0


In [18]:
df.columns

Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')

In [19]:
cls= [ 'artist_id','artist_name', 'artist_location', 'artist_latitude', 'artist_longitude'
       ]

In [20]:
artist_df = df[cls]
artist_df

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,ARD7TVE1187B99BFB1,Casual,California - LA,NaN,NaN


In [21]:
artist_df_list= artist_df.values.tolist()

In [23]:
artist_df_list[0]

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

In [26]:
artist_table_insert = ("""
    INSERT INTO artists (artist_id, name, location, latitude, longitude)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (artist_id) DO NOTHING
""")

cur.execute(artist_table_insert, artist_df_list[0])
conn.commit()

In [29]:
songs_cls=['song_id' ,'title','artist_id',  'year',  'duration']

In [30]:
songs_df= df[songs_cls]

In [31]:
songs_df_list=songs_df.values.tolist()
songs_df_list[0]

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

In [32]:

song_table_insert = ("""
    INSERT INTO songs (song_id ,title,artist_id,  year,  duration)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (song_id) DO NOTHING
""")

cur.execute(song_table_insert, songs_df_list[0])
conn.commit()

In [ ]:
log_files = get_files('data/log_data')
filepath = log_files[1]
filepath

In [37]:
df= pd.read_json(filepath, lines= True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,N.E.R.D. FEATURING MALICE,Logged In,Jayden,M,0,Fox,288.99220,free,"New Orleans-Metairie, LA",PUT,NextSong,1541033612796,184,Am I High (Feat. Malice),200,1541121934796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",101
1,None,Logged In,Stefany,F,0,White,NaN,free,"Lubbock, TX",GET,Home,1540708070796,82,None,200,1541122176796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
2,Death Cab for Cutie,Logged In,Stefany,F,1,White,216.42404,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,A Lack Of Color (Album Version),200,1541122241796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
3,Tracy Gang Pussy,Logged In,Stefany,F,2,White,221.33506,free,"Lubbock, TX",PUT,NextSong,1540708070796,82,I Have A Wish,200,1541122457796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",83
4,Skillet,Logged In,Kevin,M,0,Arellano,178.02404,free,"Harrisburg-Carlisle, PA",PUT,NextSong,1540006905796,153,Monster (Album Version),200,1541126568796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",66
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,OutKast,Logged In,Ayleen,F,3,Wise,227.52608,free,"Columbia, SC",PUT,NextSong,1541085793796,70,Ova Da Wudz,200,1541184339796,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_1 like...",71
167,Bloodhound Gang,Logged In,Kaleb,M,0,Cook,260.20526,free,"Yuba City, CA",PUT,NextSong,1540679673796,53,Uhn Tiss Uhn Tiss Uhn Tiss,200,1541187675796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54
168,Rammstein,Logged In,Isaac,M,0,Valdez,228.46649,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Adios,200,1541191023796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3
169,Gershon Kingsley,Logged In,Isaac,M,1,Valdez,146.57261,free,"Saginaw, MI",PUT,NextSong,1541078099796,112,Pop Corn,200,1541191251796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,3


In [38]:
df.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [39]:
users_data=[ 'userId','firstName',  'lastName', 'gender',
        'level']

In [40]:
users_df= df[users_data]

In [41]:
users_df_list= users_df.values.tolist()
users_df_list[1]

[83, 'Stefany', 'White', 'F', 'free']

In [54]:
users_table_insert = ("""
    INSERT INTO users (user_id,first_name, last_name, gender,
        level)
    VALUES (%s, %s, %s, %s, %s)
    ON CONFLICT (user_id) DO NOTHING
""")

cur.execute(users_table_insert, users_df_list[1])
conn.commit()

In [55]:
t = pd.to_datetime(df['ts'], unit='ms')
time_data = [t, t.dt.hour, t.dt.day, t.dt.isocalendar().week, t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-02 01:25:34.796,1,2,44,11,2018,4
1,2018-11-02 01:29:36.796,1,2,44,11,2018,4
2,2018-11-02 01:30:41.796,1,2,44,11,2018,4
3,2018-11-02 01:34:17.796,1,2,44,11,2018,4
4,2018-11-02 02:42:48.796,2,2,44,11,2018,4


In [56]:
time_table_insert = ("""
    INSERT INTO time (start_time, hour, day, week, month, year, weekday)
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON CONFLICT (start_time) DO NOTHING
""")

for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

In [57]:
song_select = ("""
    SELECT songs.song_id, artists.artist_id 
    FROM songs JOIN artists ON songs.artist_id = artists.artist_id
    WHERE songs.title=%s AND artists.name=%s AND songs.duration=%s
""")

songplay_table_insert = ("""
    INSERT INTO songplays (start_time, user_id, level, song_id, artist_id, session_id, location, user_agent)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
""")

In [58]:
def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath, lines=True)
    
    # insert artist record
    artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
    cur.execute(artist_table_insert, artist_data)
    
    # insert song record
    song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
    cur.execute(song_table_insert, song_data)

In [76]:
def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df['page']=='NextSong']

    # convert timestamp column to datetime
    t = pd.to_datetime(df['ts'], unit='ms')
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    
    # insert time data records
    time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
    column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')
    time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(users_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)

In [60]:
def process_data(cur, conn, filepath, func, topk=None):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    all_files = all_files[:topk] if topk is not None else all_files
    num_files = len(all_files)
    print('processing {} files'.format(num_files))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))

In [73]:
def main():
    
    process_data(cur, conn, filepath='data/song_data', func=process_song_file, topk=5)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file, topk=5)
    
    conn.close()

In [77]:
main()

71 files found in data/song_data
processing 5 files
1/5 files processed.
2/5 files processed.
3/5 files processed.
4/5 files processed.
5/5 files processed.
30 files found in data/log_data
processing 5 files
1/5 files processed.
2/5 files processed.
3/5 files processed.
4/5 files processed.
5/5 files processed.
